***
### Metodos

Extraido do TFS os changesets:<br>
```python
tf hist /noprompt /recursive /format:detailed /version:D"1/1/2015"~D"31/12/2015" $/fPJ/PJ/build/release > 
```
Verificar as alterações:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas  /format:unix /noprompt /version:C555472~C555471 /ignorespace      
```
Pegar arquivo na versão:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt
```

** Utilizado arquivo TFS.ipynb para conversão em CSV e Changesets.ipynb.

In [264]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt

%matplotlib inline

#arquivos = ['release_changeset_20171.csv']
arquivos = ['C:\\temp\\main_changeset.csv']
TFS = '"D:\\\TFS\\\Microsoft Visual Studio 10.0\\\Common7\\\IDE\\\\tf.exe"'

In [265]:
dfChangesets = pd.read_csv(arquivos[0], encoding='ISO-8859-1', delimiter=';', header=None, 
                 names= ["Salt", "Changeset", "Desenv", "Data", "Descricao", "File", "Filename"])
dfChangesets.head(5)

,Salt,Changeset,Desenv,Data,Descricao,File,Filename
0,263572/1,554269,anderson.albuquerque,29/12/2017,SALT 263572/1 - Correção para que o Sistema ig...,uspjCalculoPrazo.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
1,263572/1,554269,anderson.albuquerque,29/12/2017,SALT 263572/1 - Correção para que o Sistema ig...,uspjParametroCalculoPrazo.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
2,263572/1,554269,anderson.albuquerque,29/12/2017,SALT 263572/1 - Correção para que o Sistema ig...,uspjPendenciaPrazo.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
3,0/0,554260,rafael.wagner,29/12/2017,Ajuste script pois o mesmo não estava preparad...,PJ011443.DH4,/fPJ/PJ/build/main/pjDev/dbscript/PJ/oracle/PJ...
4,0/0,554260,rafael.wagner,29/12/2017,Ajuste script pois o mesmo não estava preparad...,PJ011443.DH4,/fPJ/PJ/build/main/pjDev/dbscript/PJ/sqlserver...


In [266]:
dfChangesets.shape

(200775, 7)

In [ ]:
geral = []
metodos = []

#for index, row in dfChangesets[(dfChangesets.Salt == '247275/4')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '260601/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '241736/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == 'Defect 120562')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '1/1')].iterrows():
for index, row in dfChangesets.iterrows():
    if row['File'][-3:] != 'pas':
        continue
    #print (row)
    diferenca = diff(row)
    #print (diferenca)
    linhas = line(diferenca)
    #print (linhas)
    arquivo = get(row)
    #print (arquivo[-3:])
    if os.path.isfile(arquivo):
        if arquivo[-3:] == 'pas':
            for linha in linhas:
                nome = metodo(arquivo, int(linha))
                if nome != '':
                    if not (nome in metodos):
                        metodos.append([nome, arquivo])
                    #print (nome)
    for nome, arquivo in metodos:
        #print (nome)
        geral.append([nome, row["Salt"], row["Changeset"], row["Desenv"], arquivo]) #row["File"]
#print (geral)

lines: ['unit uspPopupMenu;\n', '\n', 'interface\n', '\n', 'uses\n', '  Winapi.Windows, Winapi.Messages, System.SysUtils, System.Classes, Vcl.Graphics, Vcl.Controls, Vcl.Forms, Vcl.Dialogs,\n', '  Vcl.Menus;\n', '\n', 'type\n', '  TPopupMode = (pmStandard, pmScroll);\n', '\n', '  TspPopupMenu = class(TPopupMenu)\n', '  private\n', '    FPopupMode: TPopupMode;\n', '    FVisibleFilter: boolean;\n', '    { Private declarations }\n', '  protected\n', '    { Protected declarations }\n', '    procedure Loaded; override;\n', '  public\n', '    { Public declarations }\n', '    constructor Create(AOwner: TComponent); override;\n', '    procedure Popup(X, Y: Integer); override;\n', '  published\n', '    { Published declarations }\n', '    property PopupMode: TPopupMode read FPopupMode write FPopupMode default pmStandard;\n', '    property VisibleFilter: boolean read FVisibleFilter write FVisibleFilter default true;\n', '  end;\n', '\n', 'implementation\n', '\n', 'uses\n', '  uspVCLStylesUtils, u

lines: ['unit uspPopupMenu;\n', '\n', 'interface\n', '\n', 'uses\n', '  Winapi.Windows, Winapi.Messages, System.SysUtils, System.Classes, Vcl.Graphics, Vcl.Controls, Vcl.Forms, Vcl.Dialogs,\n', '  Vcl.Menus;\n', '\n', 'type\n', '  TPopupMode = (pmStandard, pmScroll);\n', '\n', '  TspPopupMenu = class(TPopupMenu)\n', '  private\n', '    FPopupMode: TPopupMode;\n', '    FVisibleFilter: boolean;\n', '    { Private declarations }\n', '  protected\n', '    { Protected declarations }\n', '    procedure Loaded; override;\n', '  public\n', '    { Public declarations }\n', '    constructor Create(AOwner: TComponent); override;\n', '    procedure Popup(X, Y: Integer); override;\n', '  published\n', '    { Published declarations }\n', '    property PopupMode: TPopupMode read FPopupMode write FPopupMode default pmStandard;\n', '    property VisibleFilter: boolean read FVisibleFilter write FVisibleFilter default true;\n', '  end;\n', '\n', 'implementation\n', '\n', 'uses\n', '  uspVCLStylesUtils, u

lines: ['unit uedtVisualizarModeloOuDocumento;\n', '\n', '{-------------------------------------------------------------------------------\n', 'Projeto/Sistema: SAJ / EDT\n', 'Objetivo: Visualização de modelos\n', 'Criação: dd/mm/2003 -\n', '\n', 'Alteração: 16/01/2003 - AFM\n', ' - Alterada a herança para TfedtformBotoes\n', '\n', 'Alteração: 03/12/2003 - AFM (Alessandro Fragnani de Morais)\n', '  - SALT 9391/1\n', '    - Agora posso abrir um Documento ou um Modelo\n', '\n', 'Alteração: 24/07/2006 - AFM (Alessandro Fragnani de Morais)\n', '  - SALT 20772/8\n', '    - Exibir o selo de assinatura digital\n', '\n', 'Data Modificação: 30/06/2007  Responsável: AFM (Alessandro Fragnani de Morais)\n', '  - SALT 28307/8 - documentos confidenciais\n', '-------------------------------------------------------------------------------}\n', '\n', 'interface\n', '\n', 'uses\n', '  WinApi.Windows, WinApi.Messages, System.SysUtils, System.Classes, Vcl.Graphics,\n', '  uspForm, Vcl.StdCtrls, uspLabel, 

linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['Al

lines: ['unit usajAssistente;\n', '\n', 'interface\n', '\n', 'uses\n', '  Windows, Messages, SysUtils, Classes, Graphics, Controls, Forms, Dialogs,\n', '  dxfColorButton, ExtCtrls, StdCtrls, Menus, dxfCheckBox, DB, DBClient,\n', '  usajHelpContexto, dxEdLib, dxCntner, dxExEdtr, Buttons, uspLabel, uspButtons,\n', '  uspSpeedButton, uspImage, uspButton, uspAplicacaoMetodos;\n', '\n', 'type\n', '  TChaveDlgEvento = record\n', '    tpEvento, cdChave01, cdChave02, cdChave03, flArea, cdProcedimento, cdClasse: string;\n', '  end;\n', '\n', '  TfsajHintWindow = class(THintWindow)\n', '    function CalcHintRect(MaxWidth: integer; const AHint: string; AData: Pointer): TRect; override;\n', '  end;\n', '\n', '  TfsajAssistente = class(TForm)\n', '    ibIconeD: TspSpeedButton;\n', '    ibIconeL: TspSpeedButton;\n', '    DragControl: TImage;\n', '    PopupMenuHelp: TPopupMenu;\n', '    miOQueEh: TMenuItem;\n', '    ibProximoD: TspSpeedButton;\n', '    ibProximoL: TspSpeedButton;\n', '    ibAnteriorD

...

lines: ['{**uspSelectSQL.pas\n', '  Unit do componente TspSelectMultiBanco e TspSelectSQL. }\n', 'unit uspSelectSQL;\n', '\n', 'interface\n', '\n', 'uses\n', '  System.Classes,\n', '  Bde.DBTables,\n', '  Data.DB,\n', '  DataSnap.DBClient,\n', '  uspDicionarioString,\n', '  uspFuncoesBanco,\n', '  System.Variants,\n', '  uspDicionarioObjeto,\n', '  uspQuery,\n', '  uspTipoBanco,\n', '  uspTipoAccessTypeDB,\n', '  uspTipoSpBlocoSelect,\n', '  uspSelectMultiBanco,\n', '  uspSelectSQLMetodos, uspDataset;\n', '\n', 'type\n', '  {**TspTipoUnion\n', '    Tipos de union que a classe TspSelectSQL pode realizar. }\n', '  TspTipoUnion = (tuUnionAll, tuUnion);\n', '\n', '  {**TspSelectSQL\n', '    Classe para facilitar a programação selects no servidor de\n', '    aplicação.\n', '\n', '    A classe TspSelectMultiBanco possui um conjunto de métodos\n', '    que podem ser utilizados para geração de selects\n', '    independentes do tipo de banco de dados.\n', '\n', '    Example\n', '\n', '    <PRE>

lines: ['unit uspjSolicitaSenha;\n', '\n', 'interface\n', '\n', 'uses\n', '  Windows, Messages, SysUtils, Variants, Classes, Graphics, Controls, Forms, Dialogs,\n', '  uspForm, StdCtrls, uspLabel, uspEdit, ToolWin, ComCtrls, uspToolBar, uspButton;\n', '\n', 'type\n', '  TfspjSolicitaSenha = class(TspForm)\n', '    lbSenha: TspLabel;\n', '    dfSenha: TspEdit;\n', '    bfBarraCadastro: TspToolBar;\n', '    pbEnviar: TspButton;\n', '    pbCancelar: TspButton;\n', '    procedure pbCancelarClick(Sender: TObject);\n', '    procedure pbEnviarClick(Sender: TObject);\n', '  private\n', '    FsSenhaDigitada: string;\n', '  public\n', '    property spjSenhaDigitada: string read FsSenhaDigitada;\n', '  end;\n', '\n', 'implementation\n', '\n', '{$R *.DFM}\n', '\n', '{ TfspjSolicitaSenha }\n', '\n', 'procedure TfspjSolicitaSenha.pbCancelarClick(Sender: TObject);\n', 'begin\n', '  inherited;\n', '\n', '  ModalResult := mrCancel;\n', '  Close;\n', 'end;\n', '\n', 'procedure TfspjSolicitaSenha.pbEnvia

lines: ['unit uspjDistpendenciaServ;\n', '\n', 'interface\n', '\n', 'uses\n', '  spServidor_TLB, uspEntidade, uspConjuntoDados, uspGeraCodigo, uspSelectSQL,\n', '  SysUtils, usajConstante, uspjConstante, uspjparametro, uspjEstatIntimacao,\n', '  DB, uspFuncoesSql, uspjfuncoes, uspFuncoesString, uspAtualizacao, uspjConfIntima,\n', '  uspjDistProvisoria, uspjPendenciaPrazo, uspjDistProcesso, uspjFuncoesDistrib,\n', '  uspjProcuradorHist, uspjChefia, uspjPedidoRedist, uspjNRedistDemanda, uspExcecao,\n', '  uspjPedRedistPend, uspjDemoIntimacao, uspjUsuario, esajnome, DBClient, uspFuncoes,\n', '  uspjVarDist, uspjProcurador, uspjDistProcessoServ, uspjProcesso, uspjProcessomv,\n', '  uspjEstatDist, uspjComplexintima, uspjVarDistProcur, uspjConfAudiencia, Classes, Variants,\n', '  uspjEstatAudiencia, uspjDemoAudiencia, uspClientDataSet, uspjDistpendencia,\n', '  uspjDocEmitido, usajLocal, uspjObjetoRespServ;\n', '\n', 'type\n', '  TespjDistpendenciaServ = class(TspEntidade)\n', '  private\n',

lines: ['unit uprjcFrmValoresProcesso;\n', '\n', 'interface\n', '\n', 'uses\n', '  Windows, Messages, SysUtils, Classes, Graphics, Controls, Forms, Dialogs,\n', '  uspBotoesCadastro, ExtCtrls, uspCtrlGridFiltro, Grids, DBGrids, uspCDBGrid,\n', '  uspDBGrid, uspGridFiltro, OvcPF, uspCampoMascara, OvcBase, OvcEF, OvcPB, OvcNF,\n', '  uspCampoNumerico, Cdropctl, cCal, uspDateTimeCombo, StdCtrls, uspLabel,\n', '  ComCtrls, uspPageControl, DB, DBClient, uspClientDataSet, uspDataSource, Boxes,\n', '  uspMemoBotaoEdit, uspButton, uspComboBox, uspBevel, DBCtrls, uspDBNavigator,\n', '  uspControleCadastro, uspControlePrincipal, uspPanel, uspConjuntoDados,\n', '  uprjcGrpValProc, uprjcGrpValContrib, uprjcGrpValJuro, uprjcGrpValIndex,\n', '  uprjcTipoGrupoValor, uprjcGrpValProcHist, uspDBLookupComboBox, uspInterface,\n', '  uspFuncoesClasses, usaj5PadraoObserver, uprjcFrmCalculoHistProp, usajMensagens,\n', '  uprjcTribunalJuro, uprjcConstante, uprjcTribunalIndex, uprjcMensagem,\n', '  uprjcContri

linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['Al

In [270]:
dfChangesets[(dfChangesets.Salt == '241736/1')]

,Salt,Changeset,Desenv,Data,Descricao,File,Filename
143593,241736/1,521073,douglas.zuliani,8/6/2017,SALT 241736/1 - Corrigido problema ao lançar ...,uspjProcessomvServ.pas,/fPJ/PJ/build/main/pjDev/src/pj/Servidor/uspjP...


In [ ]:
df = pd.DataFrame(geral)
df.to_csv('C:\\temp\\geral.csv', index=False, header=False, encoding ='ISO-8859-1', sep=';')

In [177]:
#File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
#===================================================================
#150,151c150
#<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
#<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
#---
#>   if nErroLogin > 0 then
#===================================================================

def diff(row):
    comando = TFS + ' difference ' + '$' + row['Filename'] + ' /format:unix /noprompt '
    comando += '/version:' + 'C' + str(row['Changeset']) + '~' + 'C' + str(int(str(row['Changeset']).replace('C', '')) - 1) + ' /ignorespace'
    #print (comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (a)
    return retorno

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas', 'Changeset':'555472'}
print (diff(teste))

teste = {'Filename':'/fPJ/PJ/build/release/4.0.18/USP/pjDev/src/3rd_party/edtDev/src/Cliente/uedtEditor5.pas', 'Changeset':'551248'}
print (diff(teste))


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
150,151c150
<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
---
>   if nErroLogin > 0 then

File: D:\UNJ\fPJ\PJ\build\release\4.0.18\USP\pjDev\src\3rd_party\edtDev\src\Cliente\uedtEditor5.pas
1411,1412d1410
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     procedure PassarParametrosParaEnvioDeEmail(oForm: TspForm); virtual;
15266,15267c15264
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     PassarParametrosParaEnvioDeEmail(oForm);
---
>     oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
21121,21126d21117
< // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
< procedure TfedtEditor5.PassarParametrosParaEnvioDeEmail(oForm: TspForm);
< begin
<   oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
< end;
< 



In [2]:
pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /format:unix /noprompt /version:C553471~C553470 /ignorespace')
a = pipe.read()
pipe.close()
print (a)


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor\uspjSelectFluxoObjetoBasico.pas
544a545
>     wflSelectPrincipal.AddColuna('p.flLitispendencia as cc_flLitispendencia');
546d546
<     //13/12/2017 - Wilian.Ivo - Salt: 255004/2
548,559c548,552
<       FSQL.Decode(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia'),
<       [QuotedStr(cLitisPendente), QuotedStr(sLITIS_PENDENTE), QuotedStr(cLitisSuspeita),
<       QuotedStr(sLITIS_SUSPEITA), QuotedStr(cLitisConfirmada), QuotedStr(sLITIS_CONFIRMADA),
<       QuotedStr(cLitisConexao), QuotedStr(sLITIS_CONEXAO), QuotedStr(cLitisDescartada),
<       QuotedStr(sLITIS_DESCARTADA), QuotedStr(sLITIS_INEXISTENTE)]) + ' as cc_deLitispendencia');
< 
<     //26/12/2017 - Wilian.Ivo - Salt: 255004/2
<     if (oParametrosConsultaObjetoFila.TestarCampoNaProjecao('cc_deLitispendencia')) then
<       wflSelectPrincipal.AddColuna(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia') +
<         ' as cc_flLitispendencia')
<   

In [308]:
#D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor:
#Replacing uspjSelectFluxoObjetoBasico.pas

def get(row):
    #pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt')
    comando = TFS + ' get $' + row['Filename'] + ' /version:' + 'C'
    comando += str(row['Changeset']) + ' /force /noprompt'
    #print(comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (retorno)
    linhas = retorno.splitlines()
    #print(len(linhas))
    caminho = ''
    arquivo = ''
    try:
        caminho = linhas[0][:len(linhas[0])-1]
        #caminho = caminho.replace('\\', '\\\\')
        arquivo = linhas[1].replace('Replacing ', '').replace('Getting ', '')
    except:
        print ('linhas:',linhas)
    return caminho + '\\' + arquivo

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas', 'Changeset':'553471'}
get(teste)

'D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas'

In [228]:
def line(diff):
    retorno = []
    for linha in diff.splitlines():
        if linha[0].isdigit():
            registro = linha.split(',')[0]
            if registro.isdigit():
                retorno.append(registro)
            else:
                registro = linha.split('c')[0]
                if registro.isdigit():
                    retorno.append(registro)
                else:
                    registro = linha.split('d')[0]
                    if registro.isdigit():
                        retorno.append(registro)
    return retorno


In [306]:
def metodo(arquivo, linha):
    #f = open('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas')
    f = open(arquivo)
    lines = f.readlines()
    if linha > len(lines):
        print ('arquivo:', arquivo)
        metodo = ''
    else:
        while (linha > 0):
            texto = ''
            try:
                texto = lines[linha]
            except:
                print ('lines:', lines)
                texto = ''
                metodo = ''
            if re.match('function.*\..*\;', texto) or re.match('procedure.*\..*\;', texto): 
                metodo = texto
                break
            linha -= 1
        else:
            metodo = ''
    return metodo.replace('\n', '')

metodo('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas', 800)

'procedure TspjSelectFluxoObjetoBasico.AdicionarCondicoes;'